In [130]:
import random
from bs4 import BeautifulSoup as bs
import time
import requests
import re
import numpy as np
import pandas as pd

In [131]:
url = requests.get('https://myanimelist.net/anime/season/archive')
soup = bs(url.text,"html5lib")


In [138]:
#picking a random set of 96 seasons to search th
seasons = soup.find_all('td')
randindex = random.sample(range(0, 150), 80)
seasons_to_pick = [seasons[x] for x in randindex]
seasons_to_pick

[<td>\n                                              <a href="https://myanimelist.net/anime/season/2000/spring">\n                  Spring 2000\n                </a>\n                          </td>,
 <td>\n                                              <a href="https://myanimelist.net/anime/season/2008/winter">\n                  Winter 2008\n                </a>\n                          </td>,
 <td>\n                                              <a href="https://myanimelist.net/anime/season/1994/fall">\n                  Fall 1994\n                </a>\n                          </td>,
 <td>\n                                              <a href="https://myanimelist.net/anime/season/1982/winter">\n                  Winter 1982\n                </a>\n                          </td>,
 <td>\n                                              <a href="https://myanimelist.net/anime/season/1996/winter">\n                  Winter 1996\n                </a>\n                          </td>,
 <td

Iterating through every show in a given season

In [183]:
"""fine tune the lookup list"""
def getText(soup,df):
    testdf = pd.Series(index=['Name','Studio','Episodes','Genres','Source','Type','Score','Season','Year'])
    for index, s in enumerate(soup.find_all('div',class_=re.compile(r'seasonal-anime-list js-seasonal-anime-list js-seasonal-anime-list-key'))):
        #grab each show in the above category
        if index is 1:
            continue
        else:
            #print (s.find_all('div',class_="seasonal-anime js-seasonal-anime")),index
            #print (soup.find('title').text.strip().split(' ')[0]), (soup.find('title').text.strip().split(' ')[1])
            for anime in s.find_all('div',class_="seasonal-anime js-seasonal-anime"):
                testdf.Season = (soup.find('title').text.strip().split(' ')[0])
                testdf.Year = (soup.find('title').text.strip().split(' ')[1])
                testdf.Name = anime.find('a',{'class' : 'link-title'}).text.strip()
                testdf.Studio = anime.find('span',{"class":'producer'}).text.strip()
                testdf.Episodes = anime.find('div',{'class':'eps'}).text.strip().split(' ')[0]
                testdf.Source = anime.find('span',{"class":'source'}).text.strip()
                testdf.Type = re.search("[a-zA-Z]+",str(anime.find('div',{'class':'info'}).text)).group(0)
                testdf.Score = anime.find('span',{'class':'score'}).text.strip()
                testdf.Genres = [x.text.strip() for x in anime.find_all('span',class_='genre')]
                df = df.append(testdf,ignore_index=True)
    return df

Iterating through <96 seasons

In [186]:
df = pd.DataFrame(columns=['Name','Studio','Episodes','Genres','Source','Type','Score','Season','Year'])
time_avg = []
for index,s in enumerate(seasons_to_pick):
    t = time.time()
    try:
        url = requests.get(s.find('a').get('href'), timeout=.9,verify=True,stream=False)
        print '{} time to grab page is {}'.format(index,time.time() - t)
        time_avg.append(time.time() - t)
        soup = bs(url.text)
        df = getText(soup,df)
    except requests.exceptions.Timeout:
        print 'taking too long, skipping'
print np.average(time_avg)

0 time to grab page is 0.871999979019
1 time to grab page is 1.09500002861
2 time to grab page is 0.90700006485
3 time to grab page is 0.721999883652
4 time to grab page is 0.927000045776
5 time to grab page is 0.702000141144
6 time to grab page is 0.766000032425
7 time to grab page is 1.03100013733
8 time to grab page is 1.18700003624
9 time to grab page is 0.883999824524
10 time to grab page is 0.740999937057
11 time to grab page is 0.958999872208
12 time to grab page is 1.17500019073
13 time to grab page is 2.22100019455
14 time to grab page is 1.46900010109
15 time to grab page is 1.04800009727
16 time to grab page is 0.378999948502
17 time to grab page is 1.12999987602
18 time to grab page is 1.15499997139
19 time to grab page is 1.49399995804
taking too long, skipping
21 time to grab page is 1.75300002098
22 time to grab page is 2.02300000191
23 time to grab page is 1.66000008583
24 time to grab page is 1.58400011063
25 time to grab page is 1.41099977493
26 time to grab page is 1

In [187]:
df

,Name,Studio,Episodes,Genres,Source,Type,Score,Season,Year
0,Love Hina,Xebec,24,"[Comedy, Ecchi, Harem, Romance, Shounen, Slice...",Manga,TV,7.27,Spring,2000
1,Yu☆Gi☆Oh! Duel Monsters,Studio Gallop,224,"[Adventure, Game, Shounen]",Manga,TV,7.58,Spring,2000
2,Ayashi no Ceres,Studio Pierrot,24,"[Adventure, Comedy, Horror, Psychological, Sup...",Manga,TV,7.36,Spring,2000
3,Gensoumaden Saiyuuki,Studio Pierrot,50,"[Adventure, Comedy, Supernatural, Drama, Shounen]",Manga,TV,7.70,Spring,2000
4,Seikai no Senki,Sunrise,13,"[Action, Military, Romance, Sci-Fi, Space]",Light novel,TV,7.84,Spring,2000
5,NieA Under 7,Triangle Staff,13,"[Comedy, Sci-Fi, Slice of Life]",Original,TV,6.97,Spring,2000
6,Boys Be...,Hal Film Maker,13,"[Slice of Life, Comedy, Romance, Shounen]",Manga,TV,6.57,Spring,2000
7,Gate Keepers,Gonzo,24,"[Action, Comedy, Fantasy, Mecha, Sci-Fi, Shounen]",Game,TV,7.06,Spring,2000
8,Daa! Daa! Daa!,J.C.Staff,78,"[Comedy, Sci-Fi, Shoujo]",Manga,TV,7.79,Spring,2000
9,Sakura Taisen,Madhouse,25,"[Adventure, Mecha, Sci-Fi, Shounen]",Game,TV,7.00,Spring,2000


this is a test function

In [172]:
#find all types of animes in a given season. i.e. TV(new), TV(cont), OVA etc
def count_iterable(i):
    return sum(1 for e in i)


dftest = pd.DataFrame(columns=['Name','Studio','Episodes','Genres','Source','Type','Score','Season','Year'])
testdf = pd.Series(index=['Name','Studio','Episodes','Genres','Source','Type','Score','Season','Year'])
for index, s in enumerate(soup.find_all('div',class_=re.compile(r'seasonal-anime-list js-seasonal-anime-list js-seasonal-anime-list-key'))):
    #grab each show in the above category
    if index is 1:
        continue
    else:
        #print (s.find_all('div',class_="seasonal-anime js-seasonal-anime")),index
        #print (soup.find('title').text.strip().split(' ')[0]), (soup.find('title').text.strip().split(' ')[1])
        for anime in s.find_all('div',class_="seasonal-anime js-seasonal-anime"):
            testdf.Season = (soup.find('title').text.strip().split(' ')[0])
            testdf.Year = (soup.find('title').text.strip().split(' ')[1])
            testdf.Name = anime.find('a',{'class' : 'link-title'}).text.strip()
            testdf.Studio = anime.find('span',{"class":'producer'}).text.strip()
            testdf.Episodes = anime.find('div',{'class':'eps'}).text.strip().split(' ')[0]
            testdf.Source = anime.find('span',{"class":'source'}).text.strip()
            testdf.Type = re.search("[a-zA-Z]+",str(anime.find('div',{'class':'info'}).text)).group(0)
            testdf.Score = anime.find('span',{'class':'score'}).text.strip()
            testdf.Genres = [x.text.strip() for x in anime.find_all('span',class_='genre')]
            dftest = dftest.append(testdf,ignore_index=True)


In [189]:
df.to_csv('randomselection.csv',encoding='utf-8')